In [1]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json
import pickle
import requests

def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 
              'https://www.googleapis.com/auth/script.projects', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file'
              ]
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds



In [2]:
def get_sheet_data(service, spreadsheet_id, range_name):
    """특정 스프레드시트의 지정된 범위에 있는 데이터를 가져옵니다."""
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    values = result.get('values', [])

    # if not values:
    #     print('No data found.')
    # else:
    #     for row in values:
    #         # Row 데이터를 출력합니다. 필요에 따라 여기서 다른 작업을 수행할 수 있습니다.
    #         print(row)

    return values
    

In [17]:
def apply_formula_to_cells(service, spreadsheet_id, this_number):
    range_name = f"발주서!J11:J"  # J열 11행부터 끝까지   

    formulas = []
    for i in range(11, 1011):  # 11행부터 1010행까지
        formula = f'=IF(LEN(A{i})=0,"", {this_number})'
        formulas.append([formula])

    value_range_body = {
        'values': formulas
    }

    request = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=range_name, valueInputOption='USER_ENTERED', body=value_range_body)
    request.execute()

In [33]:
def apply_formula_to_cells_2(service, spreadsheet_id, this_number):
    range_name = f"발주서!I11:I"  # J열 11행부터 끝까지   

    phone_number = this_number or "01051319792"

    # 번호가 '-'를 포함하면 제거
    phone_number = phone_number.replace("-", "")

    # 번호의 시작이 '0'이 아니면 '0'을 추가
    if not phone_number.startswith("0"):
        phone_number = "0" + phone_number


    formulas = []
    for i in range(11, 1011):  # 11행부터 1010행까지
        formula = f'=IF(LEN(A{i})=0,"", \"{phone_number}\")'
        formulas.append([formula])

    value_range_body = {
        'values': formulas
    }

    request = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=range_name, valueInputOption='USER_ENTERED', body=value_range_body)
    request.execute()

In [44]:
def apply_formula_to_cells_3(service, spreadsheet_id):
    range_name = f"발주서!D11:D"  # J열 11행부터 끝까지  

    formulas = []
    for i in range(11, 1011):  # 11행부터 1010행까지
        formula = f'=IF(LEN(A{i})=0,"", 1)'
        formulas.append([formula])

    value_range_body = {
        'values': formulas
    }

    request = service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=range_name, valueInputOption='USER_ENTERED', body=value_range_body)
    request.execute()

In [31]:
def apply_color_to_cell(service, SPREADSHEET_ID, sheet_id):
    # 스프레드시트 ID를 정의합니다.

    # 배경색을 변경할 색상 값을 정의합니다.
    RED = 122 / 255
    GREEN = 213 / 255
    BLUE = 146 / 255
    ALPHA = 1.0  # 투명도는 변경할 필요가 없으므로 1로 유지


    # 배경색 변경 및 텍스트 삭제를 위한 요청을 정의합니다.
   # 배경색 변경 및 J9 셀 텍스트 삭제를 위한 요청을 정의합니다.
    requests = [
        {
            # J9 셀의 텍스트를 삭제합니다.
            "updateCells": {
                "range": {
                    "sheetId": sheet_id,  # 첫 번째 시트의 ID입니다. 필요에 따라 수정하세요.
                    "startRowIndex": 8,  # J9 셀의 행 (0부터 시작)
                    "endRowIndex": 9,    # 행의 범위는 시작 행 +1
                    "startColumnIndex": 8,  # J열의 열 (0부터 시작)
                    "endColumnIndex": 9    # 열의 범위는 시작 열 +1
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {},  # 값을 빈 값으로 설정
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        },
        {
            # J9와 J10 셀의 배경색을 변경합니다.
            "repeatCell": {
                "range": {
                    "sheetId": sheet_id,
                    "startRowIndex": 8,  # J9 셀의 시작 행
                    "endRowIndex": 10,   # J10 셀의 종료 행
                    "startColumnIndex": 8,
                    "endColumnIndex": 9
                },
                "cell": {
                    "userEnteredFormat": {
                        "backgroundColor": {
                            "red": RED,
                            "green": GREEN,
                            "blue": BLUE,
                            "alpha": ALPHA
                        }
                    }
                },
                "fields": "userEnteredFormat(backgroundColor)"
            }
        }
    ]

    # 배치 업데이트 요청을 실행합니다.
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=SPREADSHEET_ID,
        body={"requests": requests}
    ).execute()

In [32]:
def apply_format(service, SPREADSHEET_ID, sheet_id):
    # 일반 텍스트 서식 변경을 위한 요청을 정의합니다.
    requests = [
        {
            "repeatCell": {
                "range": {
                    "sheetId": sheet_id,  # 첫 번째 시트의 ID입니다. 필요에 따라 수정하세요.
                    "startColumnIndex": 8,  # I열 (0부터 시작하는 인덱스)
                    "endColumnIndex": 9,    # J열 (실제로는 I열만 선택)
                    "startRowIndex": 11,     # 변경을 시작할 행
                    # "endRowIndex": x,    # x를 행의 전체 개수로 설정하여 전체 열을 선택할 수 있습니다. (생략 가능)
                },
                "cell": {
                    "userEnteredFormat": {
                        "numberFormat": {
                            "type": "TEXT",
                        }
                    }
                },
                "fields": "userEnteredFormat.numberFormat"
            }
        }
    ]

    # 배치 업데이트 요청을 실행합니다.
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=SPREADSHEET_ID,
        body={"requests": requests}
    ).execute()


In [50]:
import time
# unit_pricing_spreadsheet_id = "1oxak1ZgQi3YhbC5UKPcmlrjV9ekMt53BVkHlycBW3uQ"
#크롤링 데이터 원본
# unit_pricing_spreadsheet_id = "132EosVTtzTuqNrfJ2mDxRD-TbOHxDGdGXKwpgA92y54"
signup_sheet_id = "1Ajz2g-pChLSoies5umhPMcgTp5m2MIjhljJOviPlWCU"
worksheet_name = "셀러컨텍포인트"
# worksheet_name = '캄므커머스_데이터원본'
# arranged_worksheet_name = '캄므커머스'
range_name = "셀러컨택포인트!B265:M"



if __name__ == "__main__":
    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    value = get_sheet_data(service_sheets, signup_sheet_id, range_name)
    
    for i in value:
        time.sleep(1)
        print(i)
        if len(i) == 1:
            break
        
        if i[1] == '':
            break
            # 스프레드시트의 메타데이터를 요청하여 모든 시트의 정보를 가져옵니다.
        sheet_metadata = service_sheets.spreadsheets().get(spreadsheetId=i[9]).execute()
        sheets = sheet_metadata.get('sheets', '')

        # 시트 이름으로부터 sheetId를 찾습니다.
        sheetId = None
        for sheet in sheets:
            if sheet.get('properties', {}).get('title', '') == "발주서":
                sheetId = sheet.get('properties', {}).get('sheetId', '')
                break

        if sheetId is not None:
            # sheetId를 사용하여 요청을 구성하고 실행하는 코드를 여기에 추가합니다.
            print(f"Found sheetId for : {sheetId}")
        else:
            print(f"Sheet named not found.")

        # #형식 바꾸기
        # apply_format(service_sheets, i[9], sheetId)
        # #색상 바꾸기
        # apply_color_to_cell(service_sheets, i[9], sheetId)
        # #J열 코드
        # apply_formula_to_cells(service_sheets, i[9], i[0])
        # #전화번호 넣기
        # apply_formula_to_cells_2(service_sheets, i[9], i[8])
        apply_formula_to_cells_3(service_sheets, i[9])
        
        

        
        



['201', '김은애', '201_김은애', 'O', '', '', '', '', '01076242685', '1s0nr4vTmMgYw1tkBgNBX09rruS-Zq7Z9iMESyRZOzc8', 'https://docs.google.com/spreadsheets/d/1s0nr4vTmMgYw1tkBgNBX09rruS-Zq7Z9iMESyRZOzc8']
Found sheetId for : 1464675013
['202', '또또상회', '202_또또상회', 'O', '', 'O', '서혜진', '', '', '1NJgK1M3-J5GBpvIAcxxkvh-KCdPE7Q5i4XSpe1L8urQ', 'https://docs.google.com/spreadsheets/d/1NJgK1M3-J5GBpvIAcxxkvh-KCdPE7Q5i4XSpe1L8urQ']
Found sheetId for : 1464675013
['203', '주인없음', '203_주인없음', '', '', '', '', '', '', '1pWxAluQWZwUW9nAn5zrqbZRa2fWKjd5EKbcPiXnUuhs', 'https://docs.google.com/spreadsheets/d/1pWxAluQWZwUW9nAn5zrqbZRa2fWKjd5EKbcPiXnUuhs']
Found sheetId for : 1464675013
['204', '서원주', '204_서원주', '', '', '', '', '', '01077718326', '1Fe02qdIyeWyx8u1h5nr-CLw7qQnOBGqPWXdqiKE3pyU', 'https://docs.google.com/spreadsheets/d/1Fe02qdIyeWyx8u1h5nr-CLw7qQnOBGqPWXdqiKE3pyU']
Found sheetId for : 1464675013
['205', '프룻해피', '205_프룻해피', 'O', '', '', '', '', '01048829422', '1y7-rkU49LqdYJLNdOhG_iDsHUnYINRzIcSSYLR